### Import all libraries

In [7]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
pn.extension('tabulator')

In [132]:
df = pd.read_csv('user_data.csv')

#### create a column for monthly_income

In [129]:
df['monthly_income'] = np.random.randint(1, 11, size=len(df)) * 1000

#### create a column for credit_score
the dataset does not have a column for <b>credit_score</b>
let's generate random values for credit_score and insert into our dataset

In [131]:
credit_scores = np.random.randint(low=300, high=850, size=len(df))
df['credit_score'] = credit_scores

#### create a column for bank_balance
Similarly, we add another column for <b>bank_balance</b>
In this case if there is no leave date we insert random balance value otherwise we insert a 0

In [130]:
bank_balances = np.random.randint(low=100, high=100001, size=len(df))

#Set bank_balance to 0 where leave_date is None
bank_balances[df['leave_date'].isnull()] = 0
df['bank_balance'] = bank_balances

#### create a column for city

In [128]:
cities = ['London', 'Birmingham', 'Leeds', 'Glasgow', 'Sheffield',
          'Bradford', 'Manchester', 'Edinburgh', 'Liverpool', 'Bristol', 'Other']
#top 10 cities by population
#src: https://worldpopulationreview.com/countries/cities/united-kingdom
df['city'] = np.random.choice(cities, size=len(df))

#### create a column to check if a customer has left or not

In [63]:
df['is_active_customer'] = (df['leave_date'].isnull()).astype(int)

In [133]:
df['join_year'] = df['join_date'].str.extract(r'^(\d{4})-').astype(int)

In [134]:
df

,id,first_name,last_name,email,gender,age,months_as_a_customer,join_date,leave_date,join_year
0,1,Zquqjsp,Kzinjdx,rexydrr@hotmail.com,Male,25,74,2016-08-07,2022-09-25,2016
1,2,Juwyddl,Dkdipwi,hkpywgq@outlook.com,Female,20,45,2019-07-06,NaN,2019
2,3,Yjfjodr,Vbrtbkp,zojkhts@yahoo.com,Female,48,76,2015-03-15,2021-06-11,2015
3,4,Sukpurx,Kyqvwtk,astmqkh@yahoo.com,Male,21,69,2017-07-13,NaN,2017
4,5,Zprbudk,Ogsmodp,bcayldc@gmail.com,Female,40,12,2021-11-26,2022-12-19,2021
...,...,...,...,...,...,...,...,...,...,...
9995,9996,Huvapwm,Hgwohjd,pbpnooo@gmail.com,Female,49,10,2020-12-27,2021-11-11,2020
9996,9997,Dsosftv,Mhkpqbk,dhjqpve@outlook.com,Female,37,94,2015-06-20,NaN,2015
9997,9998,Majrzpn,Qvketpx,yemikfj@gmail.com,Female,21,18,2018-08-18,2020-03-05,2018
9998,9999,Kvwfdnw,Agxazcj,kumgfjb@outlook.com,Male,51,46,2019-05-29,NaN,2019


# creating the dashboard elements / widgets

### 1. A slider to select year

Get all the joining dates and leaving dates of customers. From those dates, we pick the max and min year values.

#### get max and min of join_date

In [11]:
join_dates = df['join_date']

In [12]:
years_join = join_dates.str[:4]
years_join = years_join.astype(int)
min_year_join = years_join.min()
max_year_join = years_join.max()

In [13]:
min_year_join = int(min_year_join)
min_year_join

2015

In [14]:
max_year_join = int(max_year_join)
max_year_join

2021

#### get max and min of leave_date

In [16]:
leave_dates = df['leave_date']

In [17]:
years_left = leave_dates.str[:4]
years_left = years_left.dropna().astype(int)
min_year_left = years_left.min()
max_year_left = years_left.max()

In [18]:
min_year_left = int(min_year_left)
min_year_left

2015

In [19]:
max_year_left = int(max_year_left)
max_year_left

2023

### the year sliders

1. join date slider

In [21]:
join_slider = pn.widgets.IntSlider(name='Join Date', start=min_year_join,end=max_year_join,value=min_year_join)
join_slider

IntSlider(end=2021, name='Join Date', start=2015, value=2015)

2. leave date slider

In [22]:
leave_slider = pn.widgets.IntSlider(name='Join Date', start=min_year_left,end=max_year_left,value=min_year_left)
leave_slider

IntSlider(end=2023, name='Join Date', start=2015, value=2015)

### create two separte dataframes for city based customer analysis
active and inactive

In [79]:
active_cust_df = df[df['is_active_customer'] == 1].groupby('city').agg({'monthly_income': 'mean', 'credit_score': 'mean'}).reset_index()
inactive_cust_df = df[df['is_active_customer'] == 0].groupby('city').agg({'monthly_income': 'mean', 'credit_score': 'mean'}).reset_index()

In [80]:
#round the values to 2 decimal places
active_cust_df = active_cust_df.round({'monthly_income': 2, 'credit_score': 2})
inactive_cust_df = inactive_cust_df.round({'monthly_income': 2, 'credit_score': 2})

In [85]:
active_cust_df

,city,monthly_income,credit_score,active_customers_avg_monthly_income,active_customers_avg_credit_score
0,Birmingham,5449.64,592.24,5449.64,592.24
1,Bradford,5607.27,578.11,5607.27,578.11
2,Bristol,5752.54,584.21,5752.54,584.21
3,Edinburgh,5766.67,574.66,5766.67,574.66
4,Glasgow,5844.44,575.35,5844.44,575.35
5,Leeds,5569.17,564.15,5569.17,564.15
6,Liverpool,5579.62,591.14,5579.62,591.14
7,London,5890.98,565.02,5890.98,565.02
8,Manchester,5279.35,580.49,5279.35,580.49
9,Other,5467.58,573.17,5467.58,573.17


In [86]:
inactive_cust_df

,city,monthly_income,credit_score,inactive_customers_avg_monthly_income,inactive_customers_avg_credit_score
0,Birmingham,5364.60,566.98,5364.60,566.98
1,Bradford,5584.64,581.48,5584.64,581.48
2,Bristol,5569.18,565.46,5569.18,565.46
3,Edinburgh,5484.65,580.67,5484.65,580.67
4,Glasgow,5632.81,569.75,5632.81,569.75
5,Leeds,5317.11,570.88,5317.11,570.88
6,Liverpool,5544.90,580.09,5544.90,580.09
7,London,5415.66,571.25,5415.66,571.25
8,Manchester,5733.45,574.45,5733.45,574.45
9,Other,5503.16,575.81,5503.16,575.81


#### rename the columns

In [83]:
active_cust_df['active_customers_avg_monthly_income'] = active_cust_df['monthly_income']
active_cust_df['active_customers_avg_credit_score'] = active_cust_df['credit_score']

In [84]:
inactive_cust_df['inactive_customers_avg_monthly_income'] = inactive_cust_df['monthly_income']
inactive_cust_df['inactive_customers_avg_credit_score'] = inactive_cust_df['credit_score']

#### drop the old columns

In [87]:
active_cust_df = active_cust_df.drop('monthly_income', axis=1)
active_cust_df = active_cust_df.drop('credit_score', axis=1)
inactive_cust_df = inactive_cust_df.drop('monthly_income', axis=1)
inactive_cust_df = inactive_cust_df.drop('credit_score', axis=1)

In [88]:
active_cust_df

,city,active_customers_avg_monthly_income,active_customers_avg_credit_score
0,Birmingham,5449.64,592.24
1,Bradford,5607.27,578.11
2,Bristol,5752.54,584.21
3,Edinburgh,5766.67,574.66
4,Glasgow,5844.44,575.35
5,Leeds,5569.17,564.15
6,Liverpool,5579.62,591.14
7,London,5890.98,565.02
8,Manchester,5279.35,580.49
9,Other,5467.58,573.17


In [89]:
inactive_cust_df

,city,inactive_customers_avg_monthly_income,inactive_customers_avg_credit_score
0,Birmingham,5364.60,566.98
1,Bradford,5584.64,581.48
2,Bristol,5569.18,565.46
3,Edinburgh,5484.65,580.67
4,Glasgow,5632.81,569.75
5,Leeds,5317.11,570.88
6,Liverpool,5544.90,580.09
7,London,5415.66,571.25
8,Manchester,5733.45,574.45
9,Other,5503.16,575.81


# graphs

In [90]:
yaxis_credit_score = pn.widgets.RadioButtonGroup(name='y-axis', options=['active','inactive'],button_type='success')

Connect data pipeline with widets

##### Convert df to interactive df. We do this to make it possible to use the our data in for interactive visualizations.

In [92]:
idf = df.interactive()

In [107]:
join_date_years = idf.join_date.str[:4]

In [116]:
type(join_slider)

panel.widgets.slider.IntSlider

In [115]:
type(join_date_years)

hvplot.interactive.Interactive

In [114]:
data_pipeline = (
    idf[
        (join_date_years <= join_slider) &
        (idf.city.isin(cities))
    ]
    .groupby(['city',idf.join_date.str[:4]])[yaxis_credit_score].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='join_date')
    .reset_index(drop=True)
)

TypeError: '<=' not supported between instances of 'str' and 'int'